In [1]:
sc.master

'yarn'

In [2]:
rdd = sc.parallelize(range(10))

In [3]:
rdd.map(lambda x: x%2).collect()

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

# 读取数据

In [10]:
host = "hadoop-master"
table = "student"
conf = {"hbase.zookeeper.quorum": host, "hbase.mapreduce.inputtable": table}
# bytes 转为python string
keyConv = "org.apache.spark.examples.pythonconverters.ImmutableBytesWritableToStringConverter"
valueConv = "org.apache.spark.examples.pythonconverters.HBaseResultToStringConverter"



In [11]:
hbase_rdd = sc.newAPIHadoopRDD("org.apache.hadoop.hbase.mapreduce.TableInputFormat",
                               "org.apache.hadoop.hbase.io.ImmutableBytesWritable",
                               "org.apache.hadoop.hbase.client.Result",
                               keyConverter=keyConv,
                               valueConverter=valueConv,
                               conf=conf)

In [12]:
count = hbase_rdd.count()

In [13]:
count

2

In [14]:
hbase_rdd.cache()

MapPartitionsRDD[8] at mapPartitions at SerDeUtil.scala:244

In [15]:
output = hbase_rdd.collect()
for k, v in output:
    print(k, v)

1 {"qualifier" : "age", "timestamp" : "1596267188546", "columnFamily" : "info", "row" : "1", "type" : "Put", "value" : "19"}
{"qualifier" : "gender", "timestamp" : "1596267181932", "columnFamily" : "info", "row" : "1", "type" : "Put", "value" : "M"}
{"qualifier" : "name", "timestamp" : "1596267173284", "columnFamily" : "info", "row" : "1", "type" : "Put", "value" : "jack"}
2 {"qualifier" : "age", "timestamp" : "1596267210165", "columnFamily" : "info", "row" : "2", "type" : "Put", "value" : "18"}
{"qualifier" : "gender", "timestamp" : "1596267203859", "columnFamily" : "info", "row" : "2", "type" : "Put", "value" : "F"}
{"qualifier" : "name", "timestamp" : "1596267196980", "columnFamily" : "info", "row" : "2", "type" : "Put", "value" : "rose"}


# 写数据


In [16]:
host = 'hadoop-master'
table = 'student'
# python string 转为bytes
keyConv = "org.apache.spark.examples.pythonconverters.StringToImmutableBytesWritableConverter"
# 字符串列表 写入
valueConv = "org.apache.spark.examples.pythonconverters.StringListToPutConverter"
conf = {"hbase.zookeeper.quorum": host,
        "hbase.mapred.outputtable": table,
        "mapreduce.outputformat.class": "org.apache.hadoop.hbase.mapreduce.TableOutputFormat",
        "mapreduce.job.output.key.class": "org.apache.hadoop.hbase.io.ImmutableBytesWritable",
        "mapreduce.job.output.value.class": "org.apache.hadoop.io.Writable"}


In [19]:
# row-key,column-family,column,value
raw_data = ['5,info,name,Rongcheng',
            '5,info,gender,M',
            '5,info,age,26',
            '6,info,name,Guanhua',
            '6,info,gender,M',
            '6,info,age,27']
write_rdd = sc.parallelize(raw_data).map(lambda x: (x[0], x.split(',')))
write_rdd.collect()

[('5', ['5', 'info', 'name', 'Rongcheng']),
 ('5', ['5', 'info', 'gender', 'M']),
 ('5', ['5', 'info', 'age', '26']),
 ('6', ['6', 'info', 'name', 'Guanhua']),
 ('6', ['6', 'info', 'gender', 'M']),
 ('6', ['6', 'info', 'age', '27'])]

write_rdd.saveAsNewAPIHadoopDataset(
    conf,
    keyConverter=None,
    valueConverter=None,
)

In [23]:
write_rdd.saveAsNewAPIHadoopDataset(conf=conf,
                                    keyConverter=keyConv, 
                                    valueConverter=valueConv)